In [15]:
#CREATING TOKENS FOR LLM

In [1]:
#Importing our dataset
with open("/content/drive/MyDrive/TOKENIZATION/the-verdict.txt","r", encoding="UTF-8") as f:
  raw_text = f.read()
print("Total numbers of characters:",len(raw_text))
print(raw_text[:99])

Total numbers of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [ ]:
#The goal is to tokenize these 20479 characters.

In [2]:
#I use the "re" library for spliting into token/individual words.
#Example:
import re
text = "Hello, this is Glen. Be free."
result = re.split(r'(\s)',text) #Here the '(\s)' splits where there is a white space.
print(result)

['Hello,', ' ', 'this', ' ', 'is', ' ', 'Glen.', ' ', 'Be', ' ', 'free.']


In [3]:
#But I want commas and fullstops to be split individually therefore:
result = re.split(r'([,.]|\s)',text)
print(result)

['Hello', ',', '', ' ', 'this', ' ', 'is', ' ', 'Glen', '.', '', ' ', 'Be', ' ', 'free', '.', '']


In [4]:
#But the white space characters are very much available and they're not required therefore:
result = [item for item in result if item.strip()] #.strip() removes whitespace.
print(result)

['Hello', ',', 'this', 'is', 'Glen', '.', 'Be', 'free', '.']


In [64]:
#Now watch me apply this tokenization scheme to the raw_text.
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',raw_text)
preprocessed = [item for item in preprocessed if item.strip()]
print(preprocessed[:30]) #Just checking the first 30 tokens.

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [65]:
print(len(preprocessed))

4690


In [66]:
#Create token IDs
all_words = sorted(set(preprocessed)) #Sorts in alphabetical order
vocabulary_size = len(all_words)
print(vocabulary_size)

1130


In [67]:
#Now the IDs
vocab =  {token:integer for integer, token in enumerate(all_words)} #Assigning each unique token with a unique number ID.
for i, item in enumerate(vocab.items()):
  print(item)
  if i >= 50:
    break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [68]:
#Instantiate a class for tokenization
class SimpleTokenizerV1:
  def __init__(self, vocab):
    self.str_to_int = vocab
    self.int_to_str = {i:s for s, i in vocab.items()} #'s' is our token 'i' is the unique id for vocab.

  def encode(self, text):
    preprocessed = re.split(r'[,.:;?_!"()\']|--|\s',text)
    preprocessed = [
        item.strip() for item in preprocessed if item.strip()
    ]
    ids = [self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self, ids):
    text = " ".join([self.int_to_str[i] for i in ids])
    #Replacing the spaces before the punctuations
    text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
    return text

In [69]:
#Let's test the class above.
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[56, 850, 988, 602, 533, 746, 1126, 596, 67, 38, 851, 1108, 754, 793]


In [70]:
#Now let's see if it can convert the token ids into text
tokenizer.decode(ids)

'It s the last he painted you know Mrs Gisburn said with pardonable pride'

In [71]:
# #Let's try to use words not in the vocabulary.
# text = "Glen, do you like tea"
# tokenizer.encode(text)
# #Of course there's a key error i.e Glen. This highlights the need to consider large and diverse training sets to extend vocabulary.

In [72]:
#Adding Special Context Tokens to handle unknown words.
#I can modify the tokenizer to add a new unknow token |unk| that handles unknown words and |endoftext| token for between unrelated text.
all_tokens = sorted(set(preprocessed))
all_tokens.extend(["<|endoftext|>","<|unk|>"])
vocab = {token:integer for integer, token in enumerate(all_tokens)}

In [73]:
len(vocab)

1132

In [74]:
#Let's see the last 5 entries in the updated vocab.
for i, item in enumerate(list(vocab.items())[-5:]):
  print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [75]:
 #Instantiate a class for tokenization
class SimpleTokenizerV2:
  def __init__(self, vocab):
    self.str_to_int = vocab
    self.int_to_str = {i:s for s, i in vocab.items()} #'s' is our token 'i' is the unique id for vocab.

  def encode(self, text):
    preprocessed = re.split(r'[,.:;?_!"()\']|--|\s', text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    preprocessed = [
        item if item in self.str_to_int
        else "<|unk|>" for item in preprocessed #Dealing with unkown values.
    ]
    ids = [self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self, ids):
    text = " ".join([self.int_to_str[s] for s in ids])
    #Replacing the spaces before the punctuations
    text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
    return text

In [76]:
#Now let's see what V2 can do.
tokenizer = SimpleTokenizerV2(vocab)

In [77]:
#Our unknowns are "Glen" and "palace".
text1 = "Glen, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1,text2))
print(text)

Glen, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [78]:
tokenizer.encode(text)

[1131, 355, 1126, 628, 975, 1130, 55, 988, 956, 984, 722, 988, 1131]

In [81]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|> do you like tea <|endoftext|> In the sunlit terraces of the <|unk|>'

In [83]:
#There are other 3 special tokens i.e:
# 1: beginning of sequence(BOS) that signifies to the LLM where a piece of content begins.
# 2: end of sequence(EOS) which is useful in concatenating multiple unrelated texts.
# 3: padding (PAD) which ensures all texts have tehe same length.
#GPT models don't rely on these three tokens except for <|endoftext|>. They also don't use <|unk|> for unknowwn words instead they break the words into subwords units through Byte Pair Encoding.